In [1]:
import numpy as np
from numpy.linalg import norm
from scipy.spatial import ConvexHull

In [19]:
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import MeshcatVisualizerParams, Role, GeometrySet, CollisionFilterDeclaration
from pydrake.geometry.optimization import CspaceFreePolytope, SeparatingPlaneOrder
from pydrake.multibody.rational import RationalForwardKinematics
from ciris_plant_visualizer import CIrisPlantVisualizer
import numpy as np

from pydrake.all import (HPolyhedron, AngleAxis,
                         VPolytope, Sphere, Ellipsoid, InverseKinematics,
                         Hyperellipsoid, Simulator, Box)
import mcubes

import visualization_utils as viz_utils

import pydrake.symbolic as sym
from pydrake.all import  TriangleSurfaceMesh, Rgba, SurfaceTriangle, Sphere
from scipy.linalg import null_space
import time

In [20]:
import numpy as np
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.multibody.parsing import Parser
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig, AddFrameTriadIllustration

In [21]:
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from pydrake.all import Rgba
from ciris_plant_visualizer import CIrisPlantVisualizer  # Assuming this is set up properly
from visualization_utils import plot_polytope
from pydrake.systems.framework import DiagramBuilder

In [14]:
import ipywidgets as widgets
from IPython.display import display

In [15]:
# meshcat = StartMeshcat()
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

# Add the robot
gripper = parser.AddModels(file_name="my_sdfs/wsg_2dof.sdf")[0]
cap = parser.AddModels(file_name="my_sdfs/bottle_cap.sdf")[0]

# Set welds
plant.WeldFrames(
    plant.world_frame(), 
    plant.GetFrameByName("base_link", cap),
    RigidTransform(RotationMatrix(), [0, 0, 0]))

p_GgraspO = [0, 0, .065]
R_GgraspO = RotationMatrix.MakeXRotation(-np.pi / 2)
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("base_wsg", gripper),
    RigidTransform(R_GgraspO, p_GgraspO))



plant.Finalize()

# Release collision constraints
inspector = scene_graph.model_inspector()


# Add visualization
meshcat_params = MeshcatVisualizerParams()
meshcat_params.role = Role.kIllustration

# TCspace
Rat_FK = RationalForwardKinematics(plant)

q_star = np.zeros(plant.num_positions())

# Create the TCspaceFreePolytopes
cspace_free_polytope = CspaceFreePolytope(
    plant, 
    scene_graph,
    SeparatingPlaneOrder.kAffine,
    q_star)

print(plant.num_positions())


visualizer = CIrisPlantVisualizer(
    plant,
    builder,
    scene_graph,
    cspace_free_polytope,
    viz_role=Role.kIllustration
)

plant_context = visualizer.plant_context
diagram = visualizer.task_space_diagram
diagram_context = visualizer.task_space_diagram_context

# Set initial configuration
plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(
    plant_context, -0.025
)
# plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(
#     plant_context, 0.025
# )
diagram.ForcedPublish(diagram_context)

# visualizer.visualize_collision_constraint(factor = 1.2, num_points = 100)
# visualizer.meshcat_cspace.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(np.pi/2), np.array([0,0,1])))

INFO:drake:Meshcat listening for connections at http://localhost:7002
INFO:drake:Meshcat listening for connections at http://localhost:7003


3


In [16]:
x_init = np.array([-3.14, 1.57, -0.054])
x_goal = np.array([3.14, -1.57, -0.054])


# Define the 8 corner points of the convex hull
x_bounds = [-np.pi, np.pi]
y_bounds = [np.pi/2, -np.pi/2]
z_bounds = [-0.055, -0.025]

z_bounds_grasp = [-0.025, -0.024]

# Generate all corner points
placement_points = np.array([[x, y, z] for x in x_bounds for y in y_bounds for z in z_bounds])
grasp_points = np.array([[x, y, z] for x in x_bounds for y in y_bounds for z in z_bounds_grasp])

# Compute the convex hull
placement_hull = ConvexHull(placement_points)
grasp_hull = ConvexHull(grasp_points)

# Get the A and b representation of the half-spaces
def get_halfspaces(hull):
    A = hull.equations[:, :-1]  # Coefficients for x, y, z
    b = -hull.equations[:, -1]  # Right-hand side values
    return A, b

A_p, b_p = get_halfspaces(placement_hull)
A_g, b_g = get_halfspaces(grasp_hull)
assert((A_p @ x_init <= b_p).all())
assert((A_p @ x_goal <= b_p).all())

test_point = np.array([-3.14, 1.57, -0.055])

print("Test point:", test_point)
# Check if the test point is inside the convex hull
is_inside = np.all(A_p @ test_point <= b_p)
print("Is the test point inside the convex placement hull?", is_inside)
is_grasp = np.all(A_g @ test_point <= b_g)
print("Is the test point inside the convex grasp hull?", is_grasp)


Test point: [-3.14   1.57  -0.055]
Is the test point inside the convex placement hull? True
Is the test point inside the convex grasp hull? False


In [17]:
import quadprog

def quadprog_solve_qp(H, l, G=None, h=None, C=None, d=None, verbose=False):
    """
    min (1/2)x' H x + l' x
    subject to  G x <= h
    subject to  C x  = d
    """
    qp_G = 0.5 * (H + H.T)  # make sure P is symmetric
    qp_a = -l
    qp_C = None
    qp_b = None
    meq = 0
    if C is not None: # if equality constraint
        if G is not None: # if both equality and inequality constraints
            qp_C = -np.vstack([C, G]).T
            qp_b = -np.hstack([d, h])
        else: # if only equality constraints
            qp_C = -C.transpose()
            qp_b = -d
        meq = C.shape[0]
    elif G is not None:  # no equality constraint, only inequality constraint
        qp_C = -G.T
        qp_b = -h
    res = quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)
    if verbose:
        return res
    return res[0]

"""
Find [x0, x1g, x2r, x3g, x4r, x5], where g is grasp and r is release
minimise sum
"""


# Define the cost function 1/2 x^T H x + l^T x = ||Cx - c||^2
def to_quadratic_form(C, c):
    return np.dot(C.T, C), -np.dot(C.T, c).flatten()

c = np.zeros((15,1))
C = np.zeros((15,18))
C[:,3:] = np.eye(15)
C[:,:-3] -= np.eye(15)

H, l = to_quadratic_form(C, c)
H += np.eye(H.shape[1]) * 1e-6 # Add a small value to the diagonal to make H positive definite

# Define the equality constraints Cx = d
C_eq = np.zeros((11, 18))
C_eq[0:3, 0:3] = np.eye(3)
C_eq[3:6, -3:] = np.eye(3)
for i in range(3):
    j = 6*i
    row = 6 + i
    C_eq[row, j] = -1
    C_eq[row, j+3] = 1
for i in range(2):
    row = 9 + i
    j = 6*(i+1) - 1
    C_eq[row, j-2] = 1
    C_eq[row, j-1] = -1
    C_eq[row, j+1] = -1
    C_eq[row, j+2] = 1

# C = (11x18)
#      |   x0   |   x1g  |   x2r  |   x3g  |   x4r  |   x5  |
#      |c  w  f |c  w  f |c  w  f |c  w  f |c  w  f |c  w  f|
#    --------------------------------------------------------
#    [[ 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],   # init position
#     [ 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],   
#     [ 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#     [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],   # goal position
#     [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
#     [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
#     [-1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],   # The cap remains in the same position from x0 to x1g (from init to first grasp)
#     [ 0, 0, 0, 0, 0, 0,-1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],   # The cap remains in the same position from x2r to x3g (from first release to second grasp)
#     [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-1, 0, 0, 1, 0, 0],   # The cap remains in the same position from x4r to x5 (from second release to goal)
#     [ 0, 0, 0, 1,-1, 0,-1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],   # The change in cap and wrist position from x1g to x2r is the same (grasp to release)
#     [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,-1, 0,-1, 1, 0, 0, 0, 0]]   # The change in cap and wrist position from x3g to x4r is the same (grasp to release)

d_eq = np.vstack([x_init.reshape(3,1), x_goal.reshape(3,1), np.zeros((5,1))]).flatten()

# Define the inequality constraints Gx <= h
G = np.zeros((156, 18))
for i in range(6):
    G[i*12:(i+1)*12, 3*i:3*(i+1)] = A_p   # Placement hull
for i in range(4):
    row = 72 + i*12
    G[row:row+12, 3*(i+1):3*(i+2)] = A_g  # Grasp hull


# Joint limits (inequality constraints)
G[120:138,:] = np.eye(18)
G[138:156,:] = -np.eye(18)
    
h = np.hstack([b_p]*6 + [b_g]*4 + [np.pi, np.pi/2, -0.025]*6 + [np.pi, np.pi/2, 0.055]*6)

checkGh = np.array([-3.77458551e-15, 1.57079633e+00, -3.00000000e-02])


# Solve the QP
x = quadprog_solve_qp(H, l, G, h, C_eq, d_eq)

print("inequality constraints check:", np.all(G @ x <= h))
hashmap = {
    0: "x0",
    1: "x1g",
    2: "x2r",
    3: "x3g",
    4: "x4r",
    5: "x5"
}
if not np.all(G @ x <= h):
    #check which points are not satisfied
    for i in range(len(x)//3):
        if not np.all(G[:,3*i:3*(i+1)] @ x[3*i:3*(i+1)] <= h):
            print("\n------------------------------------")
            print("point", hashmap[i], "not satisfied")
            new_G = G[:,3*i:3*(i+1)]
            new_x = x[3*i:3*(i+1)]

            if not np.all(new_G[:72,:] @ new_x <= h[:72]):
                print("placement constraints not satisfied")
            else:
                print("placement constraints SATISFIED")

            if not np.all(new_G[72:120,:] @ new_x <= h[72:120]):
                print("grasp constraints not satisfied")
            else:
                print("grasp constraints SATISFIED")
            
            if not np.all(new_G[120:138,:] @ new_x <= h[120:138]):
                print("upper joint limits not satisfied")
            else:
                print("upper joint limits SATISFIED")

            if not np.all(new_G[138:156,:] @ new_x <= h[138:156]):
                print("lower joint limits not satisfied")
            else:
                print("lower joint limits SATISFIED")
            

print("\nequality constraints check:", np.allclose(C_eq @ x, d_eq))



# Extract the solution
x0 = x[:3]
x1g = x[3:6]
x2r = x[6:9]
x3g = x[9:12]
x4r = x[12:15]
x5 = x[15:]
print("\nx0:", x0)
print("x1g:", x1g)
print("x2r:", x2r)
print("x3g:", x3g)
print("x4r:", x4r)
print("x5:", x5)



inequality constraints check: True

equality constraints check: True

x0: [-3.14   1.57  -0.054]
x1g: [-3.14  -1.57  -0.025]
x2r: [-2.6423308e-13  1.5700000e+00 -2.5000000e-02]
x3g: [-2.65121258e-13 -1.57000000e+00 -2.50000000e-02]
x4r: [ 3.14   1.57  -0.025]
x5: [ 3.14  -1.57  -0.054]


In [18]:
# Set initial configuration
plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(
    plant_context, -0.025
)
# plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(
#     plant_context, 0.025
# )
diagram.ForcedPublish(diagram_context)

In [10]:
from math import ceil
from time import sleep
def distance(q1,q2):    
    '''Return the euclidian distance between two configurations'''
    return np.linalg.norm(q2-q1)

def lerp(q0,q1,t):    
    return q0 * (1 - t) + q1 * t


def displayedge(q0,q1,vel=2.): #vel in sec.    
    '''Display the path obtained by linear interpolation of q0 to q1 at constant velocity vel'''
    dist = distance(q0,q1)
    duration = dist / vel    
    nframes = ceil(48. * duration)
    f = 1./48.

    for i in range(nframes + 1):
        q = lerp(q0,q1,float(i)/nframes)
        plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(plant_context, q[2])
        plant.GetJointByName("base_revolute_joint", gripper).set_angle(plant_context, q[1])
        plant.GetJointByName("cap_to_base", cap).set_angle(plant_context, -q[0])
        diagram.ForcedPublish(diagram_context)
        sleep(f)

    sleep(f*50)
    
def displaypath(path):
    for q0, q1 in zip(path[:-1],path[1:]):
        print(q0,q1)
        displayedge(q0,q1)
        

path = [x0, x1g, x2r, x3g, x4r, x5]
    
displaypath(path)

[-3.14   1.57  -0.054] [-3.14  -1.57  -0.025]


[-3.14  -1.57  -0.025] [-2.6423308e-13  1.5700000e+00 -2.5000000e-02]
[-2.6423308e-13  1.5700000e+00 -2.5000000e-02] [-2.65121258e-13 -1.57000000e+00 -2.50000000e-02]
[-2.65121258e-13 -1.57000000e+00 -2.50000000e-02] [ 3.14   1.57  -0.025]
[ 3.14   1.57  -0.025] [ 3.14  -1.57  -0.054]
